In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
# Retrieve the parameter values
job_id = dbutils.widgets.get("job_id")
usecase_id = dbutils.widgets.get("usecase_id")

# Print the parameter values
print(f"job_id: {job_id}")
print(f"usecase_id: {usecase_id}")

In [0]:
class LoadProcessedTable:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.metadata_json = {"Metadata":[]}    
    self.usecase_id = None
  
  def logger(self, message):
    print(message)
    
  def set_query_string(self, usecase_parm):
    query = f"""insert into com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
(
NGEBox_Record_ID, 
usecase_id, 
HCP_NPI,
HCP_OMNI_ID,
HCP_ZIP_CODE,
external_id,
account_priority_score_vod__c,
account_vod__c,
actioned_vod__c,
action_count_vod__c,
call_objective_clm_id_vod__c,
call_objective_from_date_vod__c,
call_obj_on_by_default_vod__c,
call_obj_record_type_vod__c,
call_objective_to_date_vod__c,
dismissed_vod__c,
dismiss_count_vod__c,
display_dismiss_vod__c,
display_mark_as_complete_vod__c,
display_score_vod__c,
email_template_id_vod__c,
email_template_vault_id_vod__c,
email_template_vod__c,
expiration_date_vod__c,
Lead_Id,
marked_as_complete_vod__c,
mark_complete_count_vod__c,
no_homepage_vod__c,
planned_call_date_vod__c,
posted_date_vod__c,
priority_vod__c,
reason_vod__c,
recordtypeid_suggestion,
record_type_name_vod__c_suggest,
suggest_external_id_vod__c,
suppress_reason_vod__c,
title_vod__c,
ownerid,
detail_group_vod__c,
driver_vod__c,
email_fragment_order_vod__c,
email_fragment_vod__c,
suggest_tag_external_id_vod__c,
product_vod__c,
record_type_name_vod__c_sug_tag,
recordtypeid_suggestion_tag,
analytics_title,
suggestion_survey_vod__c,
Processed,
Not_Sent_To_Veeva_Reason,
Sent_to_Veeva,
Created_Date,
Created_By,
HCP_OMNI_ID_OLD,
HCP_NPI_OLD,
OWNERID_OLD,
HR_Hold_Flag,
PDRP_Restriction_Flag,
TKD_Sanction_Flag,
TKD_Activity_Name__c,
TKD_Activity_Description__c,
RecordTypeId,
Start_DateTime_vod__c,
URL_VOD__C,
COLOR_VOD__C,
ICON_VOD__C,
MAIL_INSTANCE_ID,
TERRITORY_TO_ADD_VOD__C,
TERRITORY_TO_DROP_VOD__C
)
select 
NGEBox_Record_ID,
usecase_id,
HCP_NPI,
HCP_OMNI_ID,
HCP_ZIP_CODE,
external_id,
account_priority_score_vod__c,
account_vod__c,
actioned_vod__c,
action_count_vod__c,
call_objective_clm_id_vod__c,
call_objective_from_date_vod__c,
nvl(call_obj_on_by_default_vod__c,True),
nvl(call_obj_record_type_vod__c,'suggestion_vod'),
call_objective_to_date_vod__c,
dismissed_vod__c,
dismiss_count_vod__c,
nvl(display_dismiss_vod__c,'True'),
nvl(display_mark_as_complete_vod__c,'False'),
nvl(display_score_vod__c,'False'),
email_template_id_vod__c,
email_template_vault_id_vod__c,
nvl(email_template_vod__c,'False'),
expiration_date_vod__c,
Lead_Id,
marked_as_complete_vod__c,
mark_complete_count_vod__c,
nvl(no_homepage_vod__c,False) ,
planned_call_date_vod__c,
posted_date_vod__c,
nvl(priority_vod__c,'normal_vod') ,
reason_vod__c,
case when usecase_id = 514 then nvl(recordtypeid_suggestion,'012d0000000xG5zAAE') else nvl(recordtypeid_suggestion,'012d0000000xG5xAAE') end,
nvl(record_type_name_vod__c_suggest,'Insight_vod'),
suggest_external_id_vod__c,
nvl(suppress_reason_vod__c,False),
title_vod__c,
Ownerid,
detail_group_vod__c,
driver_vod__c,
email_fragment_order_vod__c,
email_fragment_vod__c,
suggest_tag_external_id_vod__c,
product_vod__c,
record_type_name_vod__c_sug_tag,
recordtypeid_suggestion_tag,
analytics_title,
suggestion_survey_vod__c,
0,
NULL,
NULL,
current_timestamp,
Created_By,
NULL,
NULL,
NULL,
0,
0, 
0,
TKD_Activity_Name__c,
TKD_Activity_Description__c,
RecordTypeId,
Start_DateTime_vod__c,
URL_VOD__C,
COLOR_VOD__C,
ICON_VOD__C,
MAIL_INSTANCE_ID,
TERRITORY_TO_ADD_VOD__C,
TERRITORY_TO_DROP_VOD__C
from com_us_alyt_ngebox.ib_NGEBox_Suggestion_Requests
where Moved_To_Processed is NULL and 
((USECASE_ID = '{usecase_parm}' and usecase_id in (select usecase_id from com_us_alyt_ngebox.metadata_usecase where usecase_id ='{usecase_parm}' )) or (USECASE_ID IS NULL OR trim(USECASE_ID) = ''));

WITH t(usecase_id, hist_retention) AS (
  SELECT USECASE_ID, hist_retention
  FROM com_us_alyt_ngebox.metadata_usecase
  WHERE USECASE_ID = '{usecase_parm}'
),
s(SUGGEST_EXTERNAL_ID_VOD__C, NGEBox_Record_ID) AS (
  SELECT p.SUGGEST_EXTERNAL_ID_VOD__C, p.NGEBox_Record_ID
  FROM (select * from com_us_alyt_ngebox.ngebox_suggestion_requests_processed where usecase_id = '{usecase_parm}') p
  LEFT JOIN com_us_lake.rep_suggestion_vod__c s ON p.SUGGEST_EXTERNAL_ID_VOD__C = s.SUGGESTION_EXTERNAL_ID_VOD__C
  WHERE (
      s.SUGGESTION_EXTERNAL_ID_VOD__C IS NULL
      OR (s.SUGGESTION_EXTERNAL_ID_VOD__C IS NOT NULL AND s.DISMISSED_VOD__C = 0)
    )
)
UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed
SET
  HCP_NPI_OLD = HCP_NPI,
  HCP_OMNI_ID_OLD = HCP_OMNI_ID,
  OWNERID_OLD = OWNERID,
  Not_Sent_To_Veeva_Reason = NULL
WHERE case when '{usecase_parm}' IN ('513') then (usecase_id = '{usecase_parm}' AND CAST(created_date AS DATE) = current_date) 
    when '{usecase_parm}' IN ('514') then (usecase_id = '{usecase_parm}' AND CAST(created_date AS DATE) = current_date AND SUGGEST_EXTERNAL_ID_VOD__C IN (SELECT SUGGEST_EXTERNAL_ID_VOD__C FROM s)) 
  else (usecase_id = '{usecase_parm}' AND CAST(created_date AS DATE) >= date_add(current_date, -(SELECT hist_retention FROM t)) AND SUGGEST_EXTERNAL_ID_VOD__C IN (SELECT SUGGEST_EXTERNAL_ID_VOD__C FROM s)) end;

WITH t (external_id, REC_CNT) as (select external_id , count(*) as REC_CNT from com_us_alyt_ngebox.ib_ngebox_suggestion_requests where  usecase_id ='{usecase_parm}' and Moved_To_Processed is null
group by external_id
having count(*) > 1 ),
s(NGEBOX_RECORD_ID,external_id,usecase_id) as (select req.NGEBOX_RECORD_ID,req.external_id,req.usecase_id from com_us_alyt_ngebox.ib_NGEBox_Suggestion_Requests req join t
on  req.external_id = t.external_id where  req.usecase_id ='{usecase_parm}') 
UPDATE com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed p
SET Not_Sent_To_Veeva_Reason = 'Duplicate Record, external_id is duplicated'
where p.usecase_id = '{usecase_parm}' AND p.NGEBOX_RECORD_ID in (select NGEBOX_RECORD_ID from s );

update com_us_alyt_ngebox.ib_NGEBox_Suggestion_Requests
set Moved_To_Processed = CURRENT_TIMESTAMP 
where  usecase_id = '{usecase_parm}' and Moved_To_Processed is null 
and (usecase_id in (select usecase_id from com_us_alyt_ngebox.metadata_usecase where usecase_id = '{usecase_parm}') or (USECASE_ID IS NULL OR trim(USECASE_ID) = ''));"""
    self.query = query
  
  def run_query_and_set_sqldf(self, usecase_id):
      self.usecase_id = usecase_id
      query_list = self.query.split(";")
      self.sql_df = []  
      for curr_query in query_list:
          if curr_query.strip() == "":
              continue
          curr_query_with_param = curr_query.replace("Usecase_parm", usecase_id)
          sql_df = sqlContext.sql(curr_query_with_param).collect()
          self.sql_df.extend(sql_df)
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp
    

In [0]:
usecase_id = dbutils.widgets.get("usecase_id")
nb_obj = LoadProcessedTable()
nb_obj.set_query_string(usecase_parm=usecase_id)
nb_obj.run_query_and_set_sqldf(usecase_id=usecase_id)
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(nb_obj.response)